<a href="https://colab.research.google.com/github/choki0715/UnLiteFlowNet-PIV/blob/master/baseline3.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!mkdir data
!unzip /content/drive/MyDrive/beef-quality-prediction/data.zip -d ./data

스트리밍 출력 내용이 길어서 마지막 5000줄이 삭제되었습니다.
  inflating: ./data/train/images/cow_1_2821.jpg  
  inflating: ./data/train/images/cow_1_2824.jpg  
  inflating: ./data/train/images/cow_1_2828.jpg  
  inflating: ./data/train/images/cow_1_283.jpg  
  inflating: ./data/train/images/cow_1_2834.jpg  
  inflating: ./data/train/images/cow_1_2835.jpg  
  inflating: ./data/train/images/cow_1_2840.jpg  
  inflating: ./data/train/images/cow_1_2841.jpg  
  inflating: ./data/train/images/cow_1_2843.jpg  
  inflating: ./data/train/images/cow_1_2847.jpg  
  inflating: ./data/train/images/cow_1_285.jpg  
  inflating: ./data/train/images/cow_1_2851.jpg  
  inflating: ./data/train/images/cow_1_2855.jpg  
  inflating: ./data/train/images/cow_1_2858.jpg  
  inflating: ./data/train/images/cow_1_2859.jpg  
  inflating: ./data/train/images/cow_1_286.jpg  
  inflating: ./data/train/images/cow_1_2860.jpg  
  inflating: ./data/train/images/cow_1_2869.jpg  
  inflating: ./data/train/images/cow_1_2871.jpg  
  inflating: ./da

In [ ]:
!pip install timm

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 431 kB 6.4 MB/s 


In [ ]:
import pandas as pd
import numpy as np
import os
from os import path as osp
import cv2
from tqdm.notebook import tqdm

import torch
from torch.utils.data import DataLoader, Dataset
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torch.optim.lr_scheduler import CosineAnnealingLR
import albumentations
import timm

import matplotlib.pyplot as plt
%matplotlib inline
import seaborn as sns

from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import StratifiedKFold

from warnings import filterwarnings
filterwarnings("ignore")

device = torch.device('cuda')

In [ ]:
timm.list_models(pretrained=True)

['adv_inception_v3',
 'bat_resnext26ts',
 'beit_base_patch16_224',
 'beit_base_patch16_224_in22k',
 'beit_base_patch16_384',
 'beit_large_patch16_224',
 'beit_large_patch16_224_in22k',
 'beit_large_patch16_384',
 'beit_large_patch16_512',
 'botnet26t_256',
 'cait_m36_384',
 'cait_m48_448',
 'cait_s24_224',
 'cait_s24_384',
 'cait_s36_384',
 'cait_xs24_384',
 'cait_xxs24_224',
 'cait_xxs24_384',
 'cait_xxs36_224',
 'cait_xxs36_384',
 'coat_lite_mini',
 'coat_lite_small',
 'coat_lite_tiny',
 'coat_mini',
 'coat_tiny',
 'convit_base',
 'convit_small',
 'convit_tiny',
 'convmixer_768_32',
 'convmixer_1024_20_ks9_p14',
 'convmixer_1536_20',
 'convnext_base',
 'convnext_base_384_in22ft1k',
 'convnext_base_in22ft1k',
 'convnext_base_in22k',
 'convnext_large',
 'convnext_large_384_in22ft1k',
 'convnext_large_in22ft1k',
 'convnext_large_in22k',
 'convnext_small',
 'convnext_tiny',
 'convnext_xlarge_384_in22ft1k',
 'convnext_xlarge_in22ft1k',
 'convnext_xlarge_in22k',
 'crossvit_9_240',
 'crossv

In [ ]:
image_size = 224
label_dict = {
    '1++': 0,
    '1+': 1,
    '1': 2,
    '2': 3,
    '3': 4
}
reversed_label_dict = {v:k for k,v in label_dict.items()}

num_classes = len(label_dict)
backbone_name = 'hrnet_w64'


batch_size = 8
n_worker = 4

init_lr = 3e-5
n_epochs = 200

In [ ]:
train_path = './data/train'
test_path = './data/test'

train_all_df = pd.read_csv(osp.join(train_path, 'grade_labels.csv'))
train_all_df['file_path'] = train_all_df.imname.apply(lambda x: osp.join(train_path, 'images', x))
train_all_df['label'] = train_all_df.grade.apply(lambda x: label_dict[x])

test_df = pd.read_csv(osp.join(test_path, 'test_images.csv'))
test_df['file_path'] = test_df.imname.apply(lambda x: osp.join(test_path, 'images', x))

print(train_all_df.shape)
print(train_all_df.head())
print(test_df.shape)
test_df.head()

(10000, 4)
             imname grade                             file_path  label
0  cow_1++_4567.jpg   1++  ./data/train/images/cow_1++_4567.jpg      0
1    cow_2_1390.jpg     2    ./data/train/images/cow_2_1390.jpg      3
2  cow_1++_2581.jpg   1++  ./data/train/images/cow_1++_2581.jpg      0
3    cow_2_1689.jpg     2    ./data/train/images/cow_2_1689.jpg      3
4    cow_3_3287.jpg     3    ./data/train/images/cow_3_3287.jpg      4
(8658, 2)


,imname,file_path
0,WuSUZJHN6t.jpg,./data/test/images/WuSUZJHN6t.jpg
1,hrua4NW4Cj.jpg,./data/test/images/hrua4NW4Cj.jpg
2,GDOHhHZJug.jpg,./data/test/images/GDOHhHZJug.jpg
3,Xewfe9T1kN.jpg,./data/test/images/Xewfe9T1kN.jpg
4,y3vLHbbHFs.jpg,./data/test/images/y3vLHbbHFs.jpg


In [ ]:
# 이미지의 shape가 (248, 200, 3)이 아닌 이미지가 있는지 확인
for i in range(train_all_df.shape[0]):
  file_path = train_all_df.iloc[i]['file_path']
  image_shape = cv2.imread(file_path).shape
  if image_shape[0] != 248 or image_shape[1] != 200 or image_shape[2] != 3:
    print(file_path, image_shape)
print('done')

done


In [ ]:
train_all_df.grade.unique()

array(['1++', '2', '3', '1+', '1'], dtype=object)

In [ ]:
train_all_df.label.value_counts()

2    2201
0    2134
1    2134
3    2090
4    1441
Name: label, dtype: int64

In [ ]:
skf = StratifiedKFold(n_splits=10, shuffle=True, random_state=139)
train_all_df['fold'] = -1
for fold, (train_idx, valid_idx) in enumerate(skf.split(train_all_df, train_all_df.label)):
    train_all_df.loc[valid_idx, 'fold'] = fold

train_all_df.head()

,imname,grade,file_path,label,fold
0,cow_1++_4567.jpg,1++,./data/train/images/cow_1++_4567.jpg,0,5
1,cow_2_1390.jpg,2,./data/train/images/cow_2_1390.jpg,3,0
2,cow_1++_2581.jpg,1++,./data/train/images/cow_1++_2581.jpg,0,6
3,cow_2_1689.jpg,2,./data/train/images/cow_2_1689.jpg,3,6
4,cow_3_3287.jpg,3,./data/train/images/cow_3_3287.jpg,4,0


In [ ]:
# transforms_train = albumentations.Compose([
#     albumentations.Resize(image_size, image_size),
#     albumentations.HorizontalFlip(p=0.5),
#     albumentations.VerticalFlip(p=0.5),
#     albumentations.Normalize()
# ])

# transforms_valid = albumentations.Compose([
#     albumentations.Resize(image_size, image_size),
#     albumentations.Normalize()
# ])

transforms_train = albumentations.Compose([
    albumentations.Resize(448, 448),
    albumentations.HorizontalFlip(p=0.5),
    albumentations.VerticalFlip(p=0.5),
    albumentations.Normalize()
])

transforms_valid = albumentations.Compose([
    albumentations.Resize(448, 448),
    albumentations.Normalize()
])

In [ ]:
class BeefDataset(Dataset):
    def __init__(self, df, mode, transform=None):
        self.df = df.reset_index(drop=True)
        self.mode = mode
        self.transform = transform
        
    def __len__(self):
        return len(self.df)
    
    def __getitem__(self, index):
        row = self.df.loc[index]
        img = cv2.imread(row.file_path)
        img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
        
        if self.transform is not None:
            res = self.transform(image=img)
            img = res['image'].transpose(2,0,1)
        
        if self.mode == 'test':
            return torch.tensor(img).float()
        else:
            return torch.tensor(img).float(), torch.tensor(row.label)

In [ ]:
class BeefModel(nn.Module):
    def __init__(self):
        super().__init__()
        self.model = timm.create_model(backbone_name, pretrained=True, num_classes=num_classes)
    
    def forward(self, x):
        return self.model(x)

In [ ]:
model = BeefModel()
model.to(device);

Downloading: "https://github.com/rwightman/pytorch-image-models/releases/download/v0.1-hrnet/hrnetv2_w64-b47cc881.pth" to /root/.cache/torch/hub/checkpoints/hrnetv2_w64-b47cc881.pth


In [ ]:
optimizer = optim.Adam(model.parameters(), lr = init_lr)
# scheduler = torch.optim.lr_scheduler.CosineAnnealingLR(optimizer, n_epochs)
criterion = nn.CrossEntropyLoss().to(device)

In [ ]:
fold_id = 0
train_df = train_all_df[train_all_df['fold'] != fold_id]
valid_df = train_all_df[train_all_df['fold'] == fold_id]

dataset_train = BeefDataset(train_df, 'train', transform = transforms_train)
dataset_valid = BeefDataset(valid_df, 'valid', transform = transforms_valid)
dataset_test = BeefDataset(test_df, 'test', transform = transforms_valid)

train_loader = torch.utils.data.DataLoader(dataset_train, batch_size=batch_size, shuffle=True, num_workers = n_worker)
valid_loader = torch.utils.data.DataLoader(dataset_valid, batch_size=batch_size, shuffle=False, num_workers = n_worker)
test_loader = torch.utils.data.DataLoader(dataset_test, batch_size=batch_size, shuffle=False, num_workers = n_worker)

In [ ]:
train_df.grade.value_counts()

1      1761
1++    1707
1+     1707
2      1672
3      1153
Name: grade, dtype: int64

In [ ]:
valid_df.grade.value_counts()

1      440
1++    427
1+     427
2      418
3      288
Name: grade, dtype: int64

In [ ]:
def train_epoch(train_loader):
    model.train()
    bar = tqdm(train_loader)
    losses = []
    for batch_idx, (images, targets) in enumerate(bar):

        images, targets = images.to(device), targets.to(device)            
        
        optimizer.zero_grad()

        logit = model(images)
        loss = criterion(logit, targets)
        loss.backward()
        optimizer.step()
        
        losses.append(loss.item())

        bar.set_description(f'loss: {loss.item():.5f}')

    loss_train = np.mean(losses)
    return loss_train

def evaluate(valid_loader):
    model.eval() #evaluation 과정에서 사용하지 않아야 하는 layer들을 알아서 off 시키도록 하는 함수
    valid_loss = 0.0
    correct = 0
    with torch.no_grad(): #파라미터 업데이트 안하기 때문에 no_grad 사용
        for images, targets in tqdm(iter(valid_loader)):
            images, targets = images.to(device), targets.to(device)

            logit = model(images)
            valid_loss += criterion(logit, targets)
            pred = logit.argmax(dim=1, keepdim=True)  #11개의 class중 가장 값이 높은 것을 예측 label로 추출
            correct += pred.eq(targets.view_as(pred)).sum().item() #예측값과 실제값이 맞으면 1 아니면 0으로 합산
    valid_acc = 100 * correct / len(valid_loader.dataset)
    print('Valid set: Loss: {:.4f}, Accuracy: {}/{} ( {:.4f}%)\n'.format(valid_loss / len(valid_loader), correct, len(valid_loader.dataset), valid_acc))
    return valid_loss, valid_acc

In [ ]:
best_acc = 0
for epoch in range(1,n_epochs+1):
  train_epoch(train_loader)
  valid_loss, valid_acc = evaluate(valid_loader)

  # 베스트 모델 저장
  if best_acc < valid_acc:
      best_acc = valid_acc
      torch.save(model.state_dict(), f'./{backbone_name}_epoch{epoch}.pth') #이 디렉토리에 best_model.pth을 저장
      print(f'epoch{epoch} Model Saved.')

  0%|          | 0/1125 [00:00<?, ?it/s]

  0%|          | 0/125 [00:00<?, ?it/s]

Valid set: Loss: 1.1966, Accuracy: 451/1000 ( 45.1000%)

epoch1 Model Saved.


  0%|          | 0/1125 [00:00<?, ?it/s]

  0%|          | 0/125 [00:00<?, ?it/s]

...

Valid set: Loss: 0.5717, Accuracy: 848/1000 ( 84.8000%)

epoch37 Model Saved.


  0%|          | 0/1125 [00:00<?, ?it/s]

  0%|          | 0/125 [00:00<?, ?it/s]

Valid set: Loss: 0.5336, Accuracy: 856/1000 ( 85.6000%)

epoch38 Model Saved.


  0%|          | 0/1125 [00:00<?, ?it/s]

In [ ]:
transforms_temp = albumentations.Compose([
    albumentations.Resize(256, 256),
    albumentations.CenterCrop(256, 256),
    albumentations.Normalize()
])

dataset_temp = BeefDataset(valid_df, 'valid', transform = transforms_temp)

temp_loader = torch.utils.data.DataLoader(dataset_temp, batch_size=batch_size, shuffle=False, num_workers = n_worker)

model = BeefModel()
model.to(device);
for i in range(18,19):
  print(i)
  model.load_state_dict(torch.load(f'/content/tf_efficientnet_b7_ns_epoch{i}_bright.pth'))
  evaluate(temp_loader)

18


  0%|          | 0/125 [00:00<?, ?it/s]

Valid set: Loss: 0.6174, Accuracy: 1738/2000 ( 86.9000%)



In [ ]:
model.load_state_dict(torch.load('/content/hrnet_w64_epoch87_bright.pth'))
evaluate(valid_loader)

  0%|          | 0/63 [00:00<?, ?it/s]

Valid set: Loss: 0.4864, Accuracy: 898/1000 ( 89.8000%)



(tensor(30.6439, device='cuda:0'), 89.8)

In [ ]:
# test code

preds = []

model.eval()
with torch.no_grad(): #파라미터 업데이트 안하기 때문에 no_grad 사용
    for images in tqdm(iter(test_loader)):
        images = images.to(device)

        logit = model(images)
        pred = logit.argmax(dim=1)  #11개의 class중 가장 값이 높은 것을 예측 label로 추출
        # print(pred.tolist())
        preds.extend(pred.tolist())

  0%|          | 0/542 [00:00<?, ?it/s]

In [ ]:
grade_preds = [reversed_label_dict[pred] for pred in preds]

submission = pd.read_csv('./data/sample_submission.csv')
temp = submission.copy()
temp['id'] = test_df.imname
temp['grade'] = grade_preds

submission = pd.merge(submission['id'], temp, on='id', how='left')
submission.to_csv('submit.csv', index=False)